In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
#Power output HQ dataset (with only wind powercolumn)
power_df = pd.read_csv("../historique-production-electricite-quebec.csv")[["Date","Wind"]]

In [4]:
#Checking duplicate 'Date' in the original dataset
power_df_strtime = power_df
print("Duplicated places:")
print(power_df_strtime[power_df_strtime['Date'].duplicated()])

#for trouble in [5476,14385,22988,28901,41485]:
#    print(f"Displaying around troublesome index: {trouble}")
#    print(power_df.loc[trouble-2:trouble+3])

Duplicated places:
                            Date     Wind
5476   2020-11-01T01:30:00-05:00  2737.00
14385  2021-11-07T01:30:00-05:00  2584.00
22988  2019-11-03T01:30:00-05:00   686.00
28901  2022-11-06T01:30:00-05:00  2576.00
41485  2023-03-12T03:30:00-04:00   142.97


In [5]:
#Format time from string to TimeStamp and converted all times to UTC
power_df['Date'] = pd.to_datetime(power_df['Date'], utc =True)
power_df['Date'] = power_df['Date'].apply(lambda x: x - pd.Timedelta(hours=5))

In [ ]:
#power_df['TimeZone'] = power_df['Date'].apply(lambda x: x.utcoffset())
#print(power_df['TimeZone'].unique())

In [6]:
power_df['Date'] = power_df['Date'].apply(lambda x: x.tz_convert(None)) #remove time-zone information

In [7]:
#Sort the dataset by date
power_df = power_df.sort_values('Date')

#Replace original 'Date' column by Year:MonthDay:Hour:00 for consistant formating with weather data
power_df['Date'] = power_df['Date'].apply(lambda x: x.floor('H'))

#Adding year and month/day in the dataset
power_df['Year'] = power_df['Date'].apply(lambda x: x.year)                     #adding a year column
power_df['MonthDay'] = power_df['Date'].apply(lambda x: x.strftime('%m-%d'))    #adding a Month/Day column

/tmp/ipykernel_8314/4099056116.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_df['Date'] = power_df['Date'].apply(lambda x: x.floor('H'))


In [8]:
#checking if the last entry in inside 2023
np.array(power_df['Date'])[-1]

np.datetime64('2023-12-31T23:00:00.000000000')

In [16]:
dupes_df = power_df[power_df['Date'].duplicated(keep=False)]
dupes_df_sorted = dupes_df.sort_values('Date')

for ts in dupes_df_sorted['Date'].unique():
    print(f"\nNearby rows for duplicated timestamp: {ts}")
    nearby = power_df[(power_df['Date'] >= ts - pd.Timedelta(hours=3)) & 
                (power_df['Date'] <= ts + pd.Timedelta(hours=2))]
    print(nearby.sort_values('Date'))


Nearby rows for duplicated timestamp: 2019-11-03 01:00:00
                     Date   Wind  Year MonthDay
17192 2019-11-02 22:00:00  795.0  2019    11-02
17193 2019-11-02 23:00:00  773.0  2019    11-02
22988 2019-11-03 01:00:00  686.0  2019    11-03
17194 2019-11-03 01:00:00  712.0  2019    11-03
22989 2019-11-03 02:00:00  702.0  2019    11-03
17195 2019-11-03 03:00:00  670.0  2019    11-03

Nearby rows for duplicated timestamp: 2020-11-01 01:00:00
                     Date    Wind  Year MonthDay
20062 2020-10-31 22:00:00  2202.0  2020    10-31
5474  2020-10-31 23:00:00  2459.0  2020    10-31
5475  2020-11-01 01:00:00  2212.0  2020    11-01
5476  2020-11-01 01:00:00  2737.0  2020    11-01
5477  2020-11-01 02:00:00  2723.0  2020    11-01
5478  2020-11-01 03:00:00  2400.0  2020    11-01

Nearby rows for duplicated timestamp: 2021-11-07 01:00:00
                     Date    Wind  Year MonthDay
31894 2021-11-06 22:00:00  2855.0  2021    11-06
31895 2021-11-06 23:00:00  2819.0  2021    11-

### The above confirms that all those duplicated values were when the time zone shifted. We would need to manually change these 5 entries.

All those 01:00 will become 00:00. But I'm not sure how to deal with the duplicate 2023-03-12 02:00:00.

In [ ]:
# We shall manually replace them here (or some better way that we need think later)


10

# Weather files

In [11]:
#File names for weather data of different farms
weather_files = os.listdir('../windfarm_weather_data')

In [ ]:
#Just wanted to check how the dataset look by looking at the first farm
farm_wdf = pd.read_csv(f"../windfarm_weather_data/{weather_files[0]}")

farm_wdf['time'] = pd.to_datetime(farm_wdf['time'], utc= True)                  #convert time to Timestamp object
farm_wdf = farm_wdf.sort_values('time')                                         #sort values using time

farm_wdf['time'] = farm_wdf['time'].apply(lambda x: x - pd.Timedelta(hours=5))  #drag the times to usual days in ET

farm_wdf['Year'] = farm_wdf['time'].apply(lambda x: x.year)                     #adding a year column
farm_wdf['MonthDay'] = farm_wdf['time'].apply(lambda x: x.strftime('%m-%d'))    #adding Month/Day as a column

#Remove data from 2024 as we don't have power data from 2024
farm_wdf = farm_wdf[farm_wdf['Year'].astype(str) != '2024']

In [ ]:
for farm_name in weather_files:
    farm_wdf = pd.read_csv(f"../windfarm_weather_data/{farm_name}")
    farm_wdf['time'] = pd.to_datetime(farm_wdf['time'])                             #convert time to Timestamp object
    farm_wdf = farm_wdf.sort_values('time')                                         #sort values using time

    farm_wdf['time'] = farm_wdf['time'].apply(lambda x: x - pd.Timedelta(hours=5))  #drag the times to usual days in ET    
    if len(farm_wdf[farm_wdf['time'].duplicated()]):
        print(f"{farm_name.replace(' wind farm hourly weather 2019-2024.csv','')} has some duplicate timestamps.")
    

### Seems like weather files does not have any duplicates and have handled time-zone shifts well. Yay. 

I'll merge the datasets after I manually clean the power data.